In [1]:
import os
import pandas as pd
import holidays

In [2]:
raw_data_path = "../data/raw"
preprocessed_data_path = "../data/preprocessed"

In [3]:
start = pd.Timestamp("2017-01-01")
end = pd.Timestamp("2023-01-30")

# Define functions

In [4]:
def to_datetime(df, col_name):
    try:
        df.index = df[col_name].apply(lambda x: pd.to_datetime(str(x), format='%d.%m.%Y', utc=True))
    except:
        df.index = df[col_name].apply(lambda x: pd.to_datetime(str(x), format='%d/%m/%Y', utc=True))    
    df.index = df.index.date
    df.index = pd.to_datetime(df.index) 
    return df.index

def sum_consumptions(df):
    return df[df.filter(like="RLM",axis=1).columns].sum(1)

def scaling(df, target, target_year=2020): 
    return (df * target / df[df.index.year==target_year].sum()).copy()

def index_alignment(df):
    df.index = pd.DatetimeIndex(df.index)
    df = df.resample("D").mean()
    return df[df.index.duplicated()==False]

def unify_index(df):
    try:
        return df.loc[(df.index >= start) & (df.index <= end)]
    except:
        return df.loc[(df.index.date >= first) & (df.index.date <= last)]

# Natural gas consumption data

## Gas consumption of RLM / large consumers

Aggregated consumption data (Trading Hub Europe): https://www.tradinghub.eu/en-gb/Publications/Transparency/Aggregated-consumption-data

In [5]:
the_new = pd.read_csv(os.path.join(raw_data_path, "THE_data.csv"), sep=";")
the_new.index = to_datetime(the_new, "Gastag")

#to numeric
the_new = the_new[the_new.columns[~the_new.columns.isin(["Gastag", "Status", "Gasday", "State"])]].fillna("0")
the_new = the_new.iloc[::-1] 
the_new = the_new.apply(lambda x: x.apply(lambda y: float(y.replace(".", "").replace(",", ""))))

#from kWh to TWh
the_new = the_new / (10**9)

Gas pool data: https://www.tradinghub.eu/de-de/Download/Archiv-GASPOOL / "Sonstiges" / "Aggregierte Verbrauchsdaten"

In [6]:
gaspool = pd.read_csv(os.path.join(raw_data_path, "gaspool.csv"), sep=";")
gaspool.index = to_datetime(gaspool, "Datum")
gaspool.drop("Datum", axis=1, inplace=True)

#from MWh to TWh
gaspool = gaspool / (10**6)

NCG data: https://www.tradinghub.eu/de-de/Download/Archiv-NetConnect-Germany / "Sonstiges" / "Aggregierte Verbrauchsdaten"

In [7]:
ncg = pd.read_csv(os.path.join(raw_data_path, "ncg.csv"), sep=";")
ncg.index = to_datetime(ncg, "DayOfUse")

ncg.drop(["DayOfUse", "Status"], axis=1, inplace=True)
ncg.drop(list(ncg.filter(like='Unit', axis=1).columns), axis=1, inplace=True)

#from kWh to TWh
ncg = ncg / (10**9)

In [8]:
the =  pd.concat([
    sum_consumptions(df) for df in [the_new, gaspool, ncg]
])
the = the.sort_index()
the = the.groupby(the.index).sum()

In [9]:
the.name = 'rlm'
the = pd.DataFrame(the)
the = the.loc[~the.index.duplicated(keep='first')]

SLP consumption data: https://www.tradinghub.eu/de-de/Veröffentlichungen/Transparenz/Aggregierte-Verbrauchsdaten

In [10]:
slp =  pd.read_excel(os.path.join(raw_data_path,"230306_Restlast ab 01.01.2018.xlsx"), index_col=0, skiprows=2, sheet_name="Daten")
slp = slp ["Restlast [kWh]"] * 10**(-9) #kWh --> TWh

negative_index = slp[slp <0].index
slp[negative_index] = (slp[negative_index - pd.Timedelta("1d")].values + slp[negative_index + pd.Timedelta("1d")].values)/2

slp = slp.to_frame().rename(columns={"Restlast [kWh]":  "slp"})
slp.name = "slp"

## Industrial gas consumption

Electricity generation from gas (ENTSO-E): https://transparency.entsoe.eu/generation/r2/actualGenerationPerGenerationUnit/show

To retrieve data via the ENTSO-E API see: https://github.com/EnergieID/entsoe-py

In [11]:
electricity_from_gas = pd.read_csv(os.path.join(raw_data_path, "electricity_from_gas.csv"), index_col=0).squeeze("columns")
electricity_from_gas.index = pd.DatetimeIndex(electricity_from_gas.index)

In [12]:
electricity_from_gas.drop_duplicates(inplace=True)

Gas consumption in electricity generation (Destatis): https://www-genesis.destatis.de/genesis//online?operation=table&code=43311-0002&bypass=true&levelindex=0&levelid=1669624814066#abreadcrumb (Genesis table code 43311-0002)

In [13]:
destatis_fuel = pd.read_excel(os.path.join(raw_data_path, "destatis_monthly_fuel.xlsx"), header = 4,index_col=[0,1])
destatis_fuel = destatis_fuel[destatis_fuel.iloc[:,0]==("Erdgas, Erdölgas")]["GJ"]
destatis_fuel.index =destatis_fuel.index.get_level_values(0).values + '-' + destatis_fuel.index.get_level_values(1) 
destatis_fuel=destatis_fuel[:"2022-Dezember"]
destatis_fuel.index = pd.date_range(f'{destatis_fuel.index[0].split("-")[0]}-{destatis_fuel.index[0].split("-")[1][:3]}-01',
             f'{destatis_fuel.index[-1].split("-")[0]}-Dec-31', 
             freq="m")
destatis_fuel = destatis_fuel*0.277778 #convert GJ to MWh

destatis_fuel = destatis_fuel / 1000000 #from MWh to TWh

destatis_fuel.name = 'destatis_fuel'

In [14]:
electricity_from_gas_scaled = pd.DataFrame()
for date in destatis_fuel.index:
    conditions = ((electricity_from_gas.index.year==date.year) & 
                  (electricity_from_gas.index.month==date.month))
    ts = electricity_from_gas[conditions] * destatis_fuel[date] / electricity_from_gas[conditions].sum()
    electricity_from_gas_scaled = pd.concat([electricity_from_gas_scaled, ts], axis=0)

In [15]:
gas_to_electricity_public = electricity_from_gas_scaled[0]
gas_to_electricity_public.name = "gas_to_electricity_public"

gas_to_electricity_public = pd.DataFrame(gas_to_electricity_public)

gas_to_electricity_public = gas_to_electricity_public.loc[~gas_to_electricity_public.index.duplicated(keep='first')]

In [16]:
industry = the['rlm'] - gas_to_electricity_public['gas_to_electricity_public']

industry.name = "industry"

industry = pd.DataFrame(industry)
industry = industry.loc[~industry.index.duplicated(keep='first')]

# Control variables / other variables

## Simulated heating profiles (when2heat: w2h)

In [17]:
w2h = pd.read_csv(os.path.join(raw_data_path, "slp.csv"), index_col=0)
w2h.index = pd.to_datetime(
    pd.to_datetime(w2h.index).date
)
w2h = w2h[w2h.index.year>=2016]

In [18]:
w2h["commercial"] = w2h[["commercial_space TWh", "commercial_water TWh"]].sum(1)
w2h["residential"] = w2h[["residential_space TWh", "residential_water TWh"]].sum(1)

In [19]:
w2h_scaled = pd.DataFrame()

scaling_map = {
    'residential': ('residential', 254),
    'commercial': ('commercial', 98),
    'decentral_chp': ('commercial', 15),
    'industry_buildings': ('commercial', 25)
}

for name_out, (name_in, target) in scaling_map.items():
    w2h_scaled[name_out] = scaling(w2h[name_in], target)
    
w2h_scaled['aggregated'] = w2h_scaled['residential'] + w2h_scaled['commercial']

In [20]:
w2h = w2h_scaled.aggregated
w2h = pd.DataFrame(w2h)
w2h = w2h.loc[~w2h.index.duplicated(keep='first')]

Ambient temperature and solar radiation: "2m temperature", and "Surface solar radiation downwards" from https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=form

In [21]:
temperature = pd.read_csv(os.path.join(raw_data_path, "temperature.csv"), index_col=0)
temperature.index = pd.DatetimeIndex(temperature.index)
temperature = temperature.rename(columns={"Temp [K]" : "temperature"})
temperature = temperature.loc[~temperature.index.duplicated(keep='first')]

solar = pd.read_csv(os.path.join(raw_data_path, "solar.csv"), index_col=0)
solar.index = pd.DatetimeIndex(solar.index)
solar = solar * 2.77778e-16 #J/d --> TWh/d
solar = solar.rename(columns={"solar [J/m^2]" : "solar"})
solar = solar.loc[~solar.index.duplicated(keep='first')]

## Economic activity

Manufacturing sector production index: https://www.destatis.de/DE/Themen/Wirtschaft/Konjunkturindikatoren/_inhalt.html

In [22]:
manufacturing_sector = pd.read_csv(os.path.join(raw_data_path, "manufacturing_sector.csv"), index_col=0, sep=";", dtype={"Originalwert":float}, decimal=",")
manufacturing_sector = manufacturing_sector.iloc[:,0] #0: original
manufacturing_sector.index = pd.to_datetime(manufacturing_sector.index, format='%d.%m.%Y')

manufacturing_sector.index = manufacturing_sector.index.to_period()
manufacturing_sector = manufacturing_sector.resample('d',convention='end').bfill()
manufacturing_sector.index = manufacturing_sector.index.to_timestamp()
manufacturing_sector.name = "manufacturing_sector"

Retail sector sales index: https://www.destatis.de/DE/Themen/Wirtschaft/Konjunkturindikatoren/Einzelhandel/keh331.html#355008

In [23]:
retail_sector = pd.read_csv(os.path.join(raw_data_path, "retail_sector.csv"), index_col=0, sep=";", dtype={"Originalwert":float}, decimal=",")
retail_sector = retail_sector.iloc[:,0] #0: original (real) 
retail_sector.index = pd.to_datetime(retail_sector.index, format='%d.%m.%Y')

retail_sector.index = retail_sector.index.to_period()
retail_sector = retail_sector.resample('d',convention='end').bfill()
retail_sector.index = retail_sector.index.to_timestamp()
retail_sector.name = "retail_sector"

Hospitality sector sales index: https://www.destatis.de/DE/Themen/Branchen-Unternehmen/Gastgewerbe-Tourismus/_Grafik/_Interaktiv/umsatz-gastgewerbe.html

In [24]:
hospitality_sector = pd.read_csv(os.path.join(raw_data_path, "hospitality_sector.csv"), index_col=0, sep=";", dtype={"Originalwert":float}, decimal=",")
hospitality_sector = hospitality_sector.iloc[:,0] #0: original (real) 
hospitality_sector.index = hospitality_sector.index.map(lambda x: pd.to_datetime(x, format='%Y.%m.%d'))

hospitality_sector.index = hospitality_sector.index.to_period()
hospitality_sector = hospitality_sector.resample('d',convention='end').bfill()
hospitality_sector.index = hospitality_sector.index.to_timestamp()
hospitality_sector.name = "hospitality_sector"

Real hourly wages: https://www-genesis.destatis.de/genesis//online?operation=table&code=62361-0010&bypass=true&levelindex=0&levelid=1677750428078#abreadcrumb

In [25]:
wages = pd.read_csv(os.path.join(raw_data_path, "income.csv"), sep=";", decimal=",")
qs = wages['Zeit'].astype(str) + f'-' + wages['2_Auspraegung_Code']
qs = qs.str.replace('UART','')
wages['Date'] = pd.PeriodIndex(qs, freq='Q')
wages.set_index(pd.PeriodIndex(wages['Date']),inplace=True)
wages = wages['VST074__Reallohnindex__1Q2022=100']
wages = wages.resample('d',convention='end').bfill()
wages.index = wages.index.to_timestamp()
wages.name = 'real_wages'

## Natural gas prices

Gas prices: https://www.destatis.de/DE/Themen/Wirtschaft/Preise/Publikationen/Energiepreise/energiepreisentwicklung-xlsx-5619001.xlsx?__blob=publicationFile 

In [26]:
prices_destatis = pd.read_excel(os.path.join(raw_data_path, "Energiepreisentwicklung.xlsx"),index_col=0, parse_dates=True)
prices_destatis.index = pd.DatetimeIndex(pd.DatetimeIndex(prices_destatis.index).date)

prices_destatis = prices_destatis.reindex(the.index, method = "ffill")
prices_destatis = prices_destatis[prices_destatis.index <= '2022-12-31']

price_industry = prices_destatis.loc[:,'Erzeugerpreis Erdgas (bei Abgabe an die Industrie)']
price_industry.name = 'price_industry'
price_households = prices_destatis.loc[:,'Verbraucherpreis Erdgas (ohne Umlage)']
price_households.name = 'price_households'

# Combine data

In [27]:
df = pd.concat(
    [slp, industry, gas_to_electricity_public, w2h, manufacturing_sector, hospitality_sector, 
     retail_sector, wages, price_industry, price_households, solar, temperature], axis=1
)

In [28]:
df = unify_index(df).copy()

In [29]:
df['index']= df.index
df['time'] = range(len(df))
df['time'] = df['time'].apply(float)
df['year'] = df["index"].apply(lambda x: x.year)
df['month'] = df['index'].apply(lambda x: x.month)

In [30]:
df['weekday'] = df['index'].apply(lambda x: x.strftime('%A'))
df['weekday_num'] = df['index'].apply(lambda x: x.weekday())
df['monday'] = df['weekday'].apply(lambda x: int(x=="Monday"))
df['friday'] = df['weekday'].apply(lambda x: int(x=="Friday"))
df['weekend'] = df['weekday'].apply(lambda x: int(x in ["Saturday", "Sunday"]))

In [31]:
df['christmas_period'] = df['index'].apply(
    lambda x: int((x.month==12)&(x.day in range(24,32)))
)

country_holidays = holidays.CountryHoliday('DE', prov='BY')
df['holiday'] = df['index'].apply(lambda x: int(x in country_holidays))
df['long_weekend'] = (
    (df['holiday'].shift(-1) + df['weekend'].shift(1) - df['weekend'] == 2) | 
    (df['weekend'].shift(-1) + df['holiday'].shift(1) - df['weekend'] == 2) 
).apply(int)

In [32]:
df.columns

Index(['slp', 'industry', 'gas_to_electricity_public', 'aggregated',
       'manufacturing_sector', 'hospitality_sector', 'retail_sector',
       'real_wages', 'price_industry', 'price_households', 'solar',
       'temperature', 'index', 'time', 'year', 'month', 'weekday',
       'weekday_num', 'monday', 'friday', 'weekend', 'christmas_period',
       'holiday', 'long_weekend'],
      dtype='object')

# Export

In [33]:
df.to_csv(os.path.join(preprocessed_data_path, "preprocessed.csv"))